In [67]:
import pandas as pd
import numpy as np

In [68]:
data = pd.read_csv('data/train_spam.csv')
data.head()

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...


# Анализ и предобработка данных

In [66]:
# data['text_type'] = data['text_type'].replace(['ham'], '0')
# data['text_type'] = data['text_type'].replace(['spam'], '1')
# data['text_type'] = data['text_type'].astype(int)
# data

In [78]:
# посмотрим на сбаллансированность данных
data.groupby(by='text_type').count()

,text
text_type,
ham,11469
spam,4809


In [79]:
# посмотрим на пропуски
data.isna().sum()

text_type    0
text         0
dtype: int64

In [86]:
# предобработаем текста

from string import punctuation

def change_text(s: str):
    s = s.strip().lower()
    # убрать всю пунктуацию
    return s

data['text'] = data['text'].apply(change_text)
data.head()

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...
